In [1]:
import torch
from torch import nn

### 参数访问

#### 目标参数

In [2]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.3279],
        [-0.3041]], grad_fn=<AddmmBackward0>)

In [3]:
# 参数字典
net[2].state_dict()

OrderedDict([('weight',
              tensor([[-0.1035, -0.1062,  0.1467, -0.1902,  0.0817, -0.0022, -0.3352,  0.2126]])),
             ('bias', tensor([-0.3061]))])

In [15]:
# 通过属性访问参数
# 参数的存储类型为torch.nn.parameter.Parameter
print(type(net[2].bias))
print('')

# 访问权重与偏差
print('Weight ' + str(net[2].weight))
print('Bias:' + str(net[2].bias))
print('')

# 直接访问值
print('Data:')
print(net[2].weight.data)
print(net[2].bias.data)
print('')

# 直接访问梯度
print('Grad')
print(net[2].weight.grad)
print(net[2].bias.grad)

<class 'torch.nn.parameter.Parameter'>

Weight Parameter containing:
tensor([[-0.1035, -0.1062,  0.1467, -0.1902,  0.0817, -0.0022, -0.3352,  0.2126]],
       requires_grad=True)
Bias:Parameter containing:
tensor([-0.3061], requires_grad=True)

Data:
tensor([[-0.1035, -0.1062,  0.1467, -0.1902,  0.0817, -0.0022, -0.3352,  0.2126]])
tensor([-0.3061])

Grad
None
None


In [16]:
# 一次性访问所有参数: 使用named_parameters方法
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


#### 从嵌套块收集参数

In [17]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())


def block2():
    net = nn.Sequential()
    for i in range(4):
        # 这里使用add_module方法添加块,其可以为每一个块指定一个名称
        net.add_module(f'block {i}', block1())
    return net


rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.0168],
        [-0.0168]], grad_fn=<AddmmBackward0>)

In [18]:
# 查看嵌套快的组织方式
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


### 参数初始化

#### 内置初始化

In [19]:
# 使用自定义函数对整个网络进行初始化
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)


net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
# 使用apply函数实现该操作(apply定义在Module中)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0007,  0.0226,  0.0128, -0.0090]), tensor(0.))

In [20]:
# 对网络中的某个模块进行初始化
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)


def init_42(m):
    if type(m) == nn.Linear:
        # 42(银河系漫游指南)
        nn.init.constant_(m.weight, 42)


net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.4303,  0.1788, -0.6655, -0.5113])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [23]:
# 直接访问data元素进行修改

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  0.5298,  0.5925,  1.4474])

#### 参数绑定

In [24]:
# 我们需要给共享层一个名称，以便可以引用它的参数。
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    # 这里第二第三个隐藏层会共享参数(实际上他们也是相同的对象,不仅仅是参数相同)
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)

tensor([[-0.3298],
        [-0.3197]], grad_fn=<AddmmBackward0>)

In [25]:
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100

# 确保它们实际上是同一个对象,而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
